In [1]:
import numpy as np
import pandas as pd
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from skimage import morphology, io, exposure, img_as_float, transform,filters
from matplotlib import pyplot as plt
import cv2
import os
%matplotlib inline

In [2]:
root = 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data'
os.chdir(root)

In [3]:
def preProcess(original,kernel = 'hist'):
  if kernel == 'hist':
    enhanced = cv2.equalizeHist(original)
  else:
    clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8,8))
    enhanced = clahe.apply(original)
  return enhanced

In [4]:
def loadData(processed):
  normalized = cv2.normalize(processed.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)
  img = np.expand_dims(normalized, -1)
  X = [img]
  X = np.array(X)
  X -= X.mean()
  X /= X.std()
  return X

In [5]:
def getMask(img):
  img = img > 0.5
  size = 0.02 * np.prod((256,256))
  img = morphology.remove_small_objects(img, size)
  img = morphology.remove_small_holes(img, size)
  img = morphology.dilation(img,morphology.square(3))
  img = filters.gaussian(img)
  mask = img > 0.5
  return mask

In [6]:
def masked(img, mask):
  lungs = np.copy(img)
  lungs[~mask] = 1
  return lungs

In [7]:
def getLungs(processed,UNet):
  shape = (256,256)
  X = loadData(processed)
  detected = UNet.predict(X)[...,0].reshape(shape)
  mask = getMask(detected)
  lungs = masked(processed, mask)
  return lungs,mask

In [8]:
def Visualize(original,processed,masks,lungs,kernel):
  plt.clf()
  #overlay = cv2.addWeighted(processed,0.6,lungs,0.5,0)
  n = len(original)
  plt.figure(figsize=(12,3*n))      
    
  for i in range(n):
    plt.subplot(n,4,i*4+1), plt.imshow(original[i],cmap='gray'),plt.title('Original')
    plt.xticks([]), plt.yticks([])
    plt.subplot(n,4,i*4+2), plt.imshow(processed[i],cmap='gray'),plt.title('Processed')
    plt.xticks([]), plt.yticks([])
    plt.subplot(n,4,i*4+3), plt.imshow(masks[i],cmap='gray'),plt.title('Mask')
    plt.xticks([]), plt.yticks([])
    plt.subplot(n,4,i*4+4), plt.imshow(lungs[i],cmap='gray'),plt.title('Lungs')
    plt.xticks([]), plt.yticks([])
  plt.savefig(root+'/'+kernel+'_'+'.png',dpi=500,bbox_inches='tight')    
  plt.show()

In [9]:
#POS
dirlist = os.listdir('sampled_negative/')
# dirlist = 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\train_split\\positive\\10.1016-slash-j.anl.2020.04.002-a.png'
model_name = 'mask/trained_model.hdf5'
UNet = load_model(model_name)  

In [10]:
#NEG
neg_dirlist = os.listdir('sampled_negative/')
# dirlist = 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\train_split\\positive\\10.1016-slash-j.anl.2020.04.002-a.png'
model_name = 'mask/trained_model.hdf5'
UNet = load_model(model_name)  

In [11]:
#FOR NEGATIVE IMAGES
original_ls = []
processed_ls = []
lungs_ls =[]
mask_ls= []
kernel = 'hist'
quality = 'low'   # low | medium | high
files = [f for f in dirlist]

for i in files:
  original = cv2.imread('sampled_negative/'+i,cv2.IMREAD_GRAYSCALE)
  shape = (256,256)
  resized = cv2.resize(original,shape,interpolation = cv2.INTER_AREA)
  processed = preProcess(resized,kernel)
  lungs,mask = getLungs(processed,UNet)
  original_ls.append(resized)
  processed_ls.append(processed)  
  lungs_ls.append(lungs)
  mask_ls.append(mask)
  cv2.imwrite('masked_images\\negative'+'\\'+i+'.png', lungs)
  # cv2.imshow('image', lungs)
  # cv2.waitKey(0)
  # cv2.destroyAllWindows()

Visualize(original_ls,processed_ls,mask_ls,lungs_ls,kernel)

KeyboardInterrupt: 

<Figure size 432x288 with 0 Axes>

ValueError: Image size of 864x453600 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 864x453600 with 1852 Axes>

In [ ]:
# #FOR NEGATIVE IMAGES
# original_ls = []
# processed_ls = []
# lungs_ls =[]
# mask_ls= []
# kernel = 'hist'
# quality = 'low'   # low | medium | high
# files = [f for f in neg_dirlist]

# for i in files:
#   original = cv2.imread('train_split/1000_neg/'+i,cv2.IMREAD_GRAYSCALE)
#   shape = (256,256)
#   resized = cv2.resize(original,shape,interpolation = cv2.INTER_AREA)
#   processed = preProcess(resized,kernel)
#   lungs,mask = getLungs(processed,UNet)
#   original_ls.append(resized)
#   processed_ls.append(processed)  
#   lungs_ls.append(lungs)
#   mask_ls.append(mask)
#   cv2.imwrite('mask\\neg_mask'+'\\'+i+'.png', lungs)
#   # cv2.imshow('image', lungs)
#   # cv2.waitKey(0)
#   # cv2.destroyAllWindows()

# Visualize(original_ls,processed_ls,mask_ls,lungs_ls,kernel)

In [12]:
cv2.imshow('image', original)
cv2.waitKey(0)
cv2.destroyAllWindows()